In [11]:
import numpy as np # 필요 라이브러리 불러오기
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from collections import Counter
import pytagcloud
import plotly
import plotly.graph_objs as go

ModuleNotFoundError: No module named 'pytagcloud'

In [14]:
# 데이터 불러오기
news = pd.read_csv("./korean_news_topic_201711.csv", engine='python')
news = news['토픽키워드'] # 키워드 추출

KeyError: '토픽키워드'

In [23]:
vect = CountVectorizer() # bag of word 만들기
vect.fit(news)
bagOfWord = vect.transform(news).toarray()
query = vect.transform([news[0]]).toarray()[0]

In [4]:
jaccard_sim =[] # jaccard similarity를 이용하여 쿼리와 문장의 유사도 계산
cosine_sim = [] # cosine similarity를 이용하여 쿼리와 문장의 유사도 계산
euclidean_dist = [] # euclidean dist를 이용하여 쿼리와 문장의 유사도 계산
pcc_sim = [] # PCC를 이용하여 쿼리와 문장의 유사도 계산
for temp_news in bagOfWord:
    jaccard_sim.append(jaccard_similarity_score(query, temp_news))
    cosine_sim.append(cosine_similarity([query], [temp_news])[0][0])
    euclidean_dist.append(euclidean_distances([query], [temp_news])[0][0])
    pcc_sim.append(np.corrcoef(query, temp_news)[0, 1])

In [5]:
jaccard_sim = np.asarray(jaccard_sim) # similarity를 기반으로 정렬한 결과 저장
jaccard_sim = np.argsort(jaccard_sim)
jaccard_sim = jaccard_sim[::-1]

cosine_sim = np.asarray(cosine_sim)
cosine_sim = np.argsort(cosine_sim)
cosine_sim = cosine_sim[::-1] #역순을 위함 기본정렬이 오름차순이기 때문에

euclidean_dist = np.asarray(euclidean_dist)
euclidean_dist = np.argsort(euclidean_dist)

pcc_sim = np.asarray(pcc_sim) # similarity를 기반으로 정렬한 결과 저장
pcc_sim = np.argsort(pcc_sim)
pcc_sim = pcc_sim[::-1]

In [6]:
top10 = cosine_sim[1:11] # cosine_sim similarity, Top 10 뉴스 특성 분석
top10_news = news[top10]
top10_news

44     호조,3대,지수,실적,사상,마감,애플,훈풍,실적호조,뉴욕증시,실업률,557.06과,...
100    경기,소비,흐름,반등,기재부,훈풍,내수,회복,소비자심리지수,3분기,선구매,음식료품,...
199    폐기물,고형폐기물,형폐기물,자원화,고형,논의,에너지,환경,포럼,웰크론한텍,환경개선,...
32     거래,택자,절벽,다주택자,주택,다주,이사,가을,거래절벽,이달,취재,생각,영향,한동안...
278    시황,고점,코스닥,전환,서울경제,톡스타,하락,추락,고점논란,동반,메모리,코스피,논란...
243    트럭,전기트럭,테슬라,300만,최소,1억,신기술,책정,세미,신형,1주일,25만,이달...
18     수익률,누적수익률,누적,6.8%,투증,8%,포트폴리오,1위,평균,예적금,일임형,품질...
213    포항,처리,확대,공개,건축주,건물,범위,내진능력,능력,시공,축소,지진,국회,법안,계...
70     코스닥,장밋빛,정책,약발,급물살,전망,투자가이드,활성화,850선,제약,증권사,신라젠...
82     산업훈장,동탑산업훈장,이영호,대표,롯데푸드,수상,훈장,에너지,대상,대표이사,에너지효...
Name: 토픽키워드, dtype: object

In [ ]:
keywords =[]
for n in top10_news:
    keywords = keywords + n.split(',')

count = Counter(keywords) # 각 단어 별 개수 계산하기
tags = count.most_common(200) # 상위 200개 추출

word = []
counts = []

for key in tags:
    word.append(key[0])
    counts.append(key[1])

In [7]:
# histogram 만들기
data = [go.Bar(x=word, y=counts)]
filename = 'doc-histogram-cosine'
plotly.offline.plot(data, filename=filename + '.html')

'file://C:\\Users\\user19\\Tidus\\Similarity\\searchInfo\\doc-histogram-cosine.html'

In [8]:
# wordcloud 만들기
taglist = pytagcloud.make_tags(tags, maxsize=30) 
pytagcloud.create_tag_image(taglist, 'doc-cosine.jpg', fontname='Malgun Gothic', rectangular=False)